# **三、云平台其它编译语言使用教学——Quingo**

---

# 教学大纲
## 3. [基于Quingo的首个量子程序提交](#jump_3)
### &nbsp;&nbsp;&nbsp;&nbsp; 3.1 [编写isQ的量子程序](#jump_3_1)
### &nbsp;&nbsp;&nbsp;&nbsp; 3.2 [编译程序，输出QCIS线路](#jump_3_2)
### &nbsp;&nbsp;&nbsp;&nbsp; 3.3 [将程序提交到量子计算云平台(同QCIS流程)](#jump_3_3)

# 3 基于Quingo的首个量子程序提交<a id="jump_3"></a>
如果需要在云平台量子计算机上使用其他量子语言提交实验，思路如下：首先利用所选编程语言，在用户本地调用语言编译器，将线路编译/输出为QCIS语言，然后按照QCIS语言进行实验提交。  
（云平台JupyterLab环境已经预制好多种语言环境并且都无需安装，如有其他需求，可以联系客服。）

## 3.1 编写Quingo量子程序<a id="jump_3_1"></a>

根据Quingo语法规则，编写Quingo的量子程序。  
Quingo编程教程请参见xxxx  
注意Quingo的比特编号由0开始，而云端芯片比特编号由1开始。  
注意0->1这类编号的对应关系变化，以保证按拓扑结构设计的程序不至于拓扑结构出错。

In [ ]:
#该部分代码借鉴于Quingo的示例代码。
Quingo_code='''
import std_qcis
operation CNOT(q1: qubit, q2: qubit) : unit {
    Y2M(q2);
    CZ(q1, q2);
    Y2P(q2);
}


operation bell_state() : unit {
    using(q0: qubit, q1: qubit) {
        H(q0);
        CNOT(q0, q1);
        measure(q0);
        measure(q1);
    }
}
'''
f = open("./kernel.qu",'w')
f.write(Quingo_code)
f.close()

## 3.2 编译程序，输出QCIS线路<a id="jump_3_2"></a>

运行host.py对Quingo程序进行编译。  
Quingo源程序默认保存在与host.py同目录下的kernel.qu程序。  
如果不是JupyterLab环境，可以使用以下命令进行编译：  
>python host.py

In [ ]:
from quingo import quingo_interface as qi

qi.set_compiler('mlir')
#设置Quingo编译器
if qi.connect_backend('pyqcisim_quantumsim') is False:
    exit(-1)
#连接编译器后端
qi.call_quingo("kernel.qu", "bell_state")
#调用编译服务，编译成功，输出True，结果在当前目录下的build目录内，"线路名.qcis"文件。
#编译不成功，请根据实际报错信息进行程序调整。

#以下为仿真器结果显示
#res = qi.read_result()
#print(res)
#读取并显示仿真器结果

编译成功后，QCIS指令集被输出在源程序所在目录下的build目录，名字为：线路名.qcis

读取Quingo编译输出的QCIS文件

In [ ]:
f = open("./build/bell_state.qcis",'r')
quingo_qcis=f.read()#确保读取长度超过文件总长度。
f.close()
print(quingo_qcis)

## 3.3 将程序提交到量子计算云平台(同QCIS流程)<a id="jump_3_3"></a>

将编译出的QCIS线路提交到量子计算云平台。

In [ ]:
from ezQpy import * 

account = Account(login_key='2p1adksE+s6ib6uJa5/P/Mh4xbb4JrQAL9ym4QWtS1w=', machine_name='应答机A')
#设置用户SDK密钥，选择量子计算机，并创建实例，运行成功后，提示登录成功。

query_id_quingo = account.submit_job(circuit=quingo_qcis, version="Bell_state_quingo")


等待代码在量子计算机上完成，并读取实验结果，处理保存实验结果。

In [ ]:
if query_id_quingo:
    result=account.query_experiment(query_id_quingo, max_wait_time=360000)
    #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
    print(result)
    #后继数据应用实现。